Автор блокнота: Олег Дудник

Мануал по SQL: http://swcarpentry.github.io/sql-novice-survey/10-prog/index.html

In [ ]:
# Cell #1
import sqlite3
conn = sqlite3.connect('restaurant.db') # Создаем соединение с нашей базой данных

# Часть №1

#### Хотим таблицу Склад:

| Название ингредиента | Количество | Дата* |
| -------------------- |:----------:|:-----:|
| Брокколи             | 25         | 02.12 |
| Куриное филе         | 5          | 03.12 |
| Брокколи             | 30         | 04.12 |

*когда испортиться или когда привезли

#### Как ее правильно реализовать в реляционных базах данных?
###### Таблица Склад:  
   
| id | id ингредиента | Количество | Дата* |
| -- | -------------- |:----------:|:-----:|
| 1  | 5              | 25         | 1575314617 |
| 2  | 7              | 5          | 1575401017 |
| 3  | 5              | 30         | 1575487417 |

###### Таблица Ингредиенты:

| id ингредиента | Название |
| -- | -------------- |
| 5  | Брокколи       |
| 7  | Куриное филе   |

*unix-time

Почему именно так?

In [ ]:
# Cell #2
import sys
print(sys.getsizeof(318))
print(sys.getsizeof('Брокколи'))
d = sys.getsizeof('Брокколи') - sys.getsizeof(318)
print(d)

Создадим таблицы в нашей БД:

In [ ]:
# Cell #3
# Создаем курсор - это специальный объект который делает запросы и получает их результаты
c = conn.cursor()

# Создаем таблицу ингредиентов
c.execute('''CREATE TABLE ingredients
             (id INTEGER PRIMARY KEY, name TEXT)''')

# Создаем таблицу для склада
c.execute('''CREATE TABLE warehouse
             (id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ingredient_id INTEGER, count INTEGER, date REAL)''')

# Save (commit) the changes
conn.commit()

Заполним таблицу ингредиентов:

In [ ]:
# Cell #4
ingredients = ['Молоко', 'Яйцо', 'Мука', 'Разрыхлитель', 'Масло растительное', 'Сахар', 'Соль']

for i in map(lambda it: (it, ), ingredients):
    print(i)
    c.execute("INSERT INTO ingredients (name) VALUES (?)", i)

Посмотрим на неё:

In [ ]:
# Cell #5
c.execute("SELECT * FROM ingredients")
results = c.fetchall()
results

In [ ]:
# Cell #6
c.execute("SELECT name FROM ingredients")
results = c.fetchall()
results

In [ ]:
# Cell #7
c.execute("SELECT * FROM ingredients WHERE id=5")
print(c.fetchall())

c.execute("SELECT * FROM ingredients WHERE name='Соль'")
print(c.fetchall())

Посчитаем количество строк этой таблицы:

In [ ]:
# Cell #8
c.execute("SELECT COUNT(*) FROM ingredients")
print(c.fetchall())

c.execute("SELECT SUM(id) FROM ingredients")
print(c.fetchall())

Создадим случайные данные для Склада и запишем их в БД:

In [ ]:
# Cell #9
c.execute("SELECT id FROM ingredients")
results = c.fetchall()
print(results)
ids = list(map(lambda it: it[0], results))
print(ids)

In [ ]:
# Cell #10
import random
import datetime

for i in range(1, 28):
    count = random.randint(10, 100)
    date = datetime.datetime(2019, 11, i, hour=random.randint(12, 18), minute=random.randint(0, 59), second=0, microsecond=0, tzinfo=None)
    ingredient = random.choice(ids)
    c.execute("INSERT INTO warehouse (ingredient_id, count, date) VALUES (?, ?, ?)", (ingredient, count, date.timestamp()))

In [ ]:
# Cell #11
c.execute("SELECT * FROM warehouse")
results = c.fetchall()
results

Приведем все в нормальный вид:

In [ ]:
# Cell #12
def prettyWarehouse(cursor):
    cursor.execute("""SELECT name, count, date FROM warehouse
          JOIN ingredients WHERE warehouse.ingredient_id = ingredients.id""")
    return cursor.fetchall()

prettyWarehouse(c)

Посчитаем количество ингредиентов на складе:

In [ ]:
# Cell #13
def ingredientsOnWarehouse(cursor):
    cursor.execute("""SELECT name, SUM(count) FROM warehouse 
          JOIN ingredients WHERE warehouse.ingredient_id = ingredients.id
          GROUP BY name""")
    return cursor.fetchall()

ingredientsOnWarehouse(c)

Создадим функцию для поставки ингредиентов:

In [ ]:
# Cell #14
c.execute("SELECT id FROM ingredients WHERE name=?", ('Мука',))
results = c.fetchall()
print(results)
ingredient_id = results[0][0]
ingredient_id

In [ ]:
# Cell #15
c.execute("SELECT id FROM ingredients WHERE name=?", ('Авокадо',))
results = c.fetchall()
print(results)
print(len(results))

In [ ]:
# Cell #16
def getIngredientId(cursor, name):
    cursor.execute("SELECT id FROM ingredients WHERE name=?", (name, ))
    results = cursor.fetchall()
    if(len(results) > 0):
        return results[0][0]
    else:
        cursor.execute("INSERT INTO ingredients (name) VALUES (?)", (name, ))
        return getIngredientId(cursor, name)

In [ ]:
# Cell #17
import datetime
def supply(cursor, ingredient_name, count, date):
    ingredient_id = getIngredientId(cursor, ingredient_name)
    cursor.execute("INSERT INTO warehouse (ingredient_id, count, date) VALUES (?, ?, ?)", 
                       (ingredient_id, count, date.timestamp()))

In [ ]:
# Cell #18
c.execute("DELETE FROM warehouse");
c.execute("SELECT * FROM warehouse")
results = c.fetchall()
results

In [ ]:
# Cell #19
c.execute("SELECT * FROM ingredients")
results = c.fetchall()
results

In [ ]:
# Cell #20
supply(c, 'Мука', 10, datetime.datetime.now())
supply(c, 'Авокадо', 10, datetime.datetime.now())

In [ ]:
# Cell #21
prettyWarehouse(c)

In [ ]:
# Cell #22
c.execute("SELECT * FROM ingredients")
results = c.fetchall()
results

# Часть №2

#### Хотим таблицу Блюд:
   
| id | Название            | Ингредиенты |
| -- | ------------------- |:-----------:|
| 1  | Панкейки            | ?           |
| 2  | Буррито             | ?           |
| 3  | Овощи по деревенски | ?           |


#### Как ее правильно реализовать в реляционных базах данных?

###### Таблица Блюд:

| id | Название            |
| -- | ------------------- |
| 1  | Панкейки            |
| 2  | Буррито             |
| 3  | Овощи по деревенски |

###### Таблица Ингредиентов:

| id ингредиента | Название |
| -- | -------------- |
| 5  | Брокколи       |
| 7  | Куриное филе   |
| 12 | Молоко         |

###### Таблица Блюд-Ингредиентов:

| id Блюда | id Ингредиента | Количество ингредиентов |
| -------- | -------- | -------- |
| 1        | 12       | 250 |
| 2        | 7       | 50 |
| 3        | 5       | 10 |

In [ ]:
# Cell #23
c.execute("DELETE FROM ingredients");
c.execute("DELETE FROM warehouse");

In [ ]:
# Cell #24
# Создаем таблицу блюд
c.execute('''CREATE TABLE dishes
             (id INTEGER PRIMARY KEY, name TEXT, recipe INTEGER)''')

# Создаем таблицу для блюд-ингредиетов
c.execute('''CREATE TABLE dishes_ingredients
             (dish_id INTEGER, ingredient_id INTEGER, ingredient_count INTEGER)''')

conn.commit()

Напишем функцию для добавления блюд:

In [ ]:
# Cell #25
dishes = {
    'Панкейки': {
        'Молоко': 210,
        'Яйцо': 1,
        'Мука': 200,
        'Разрыхлитель': 5,
        'Масло растительное': 2,
        'Сахар': 30,
        'Соль': 1
    },
    
    'Буррито': {
        'Тортилья': 4,
        'Куриное филе': 12,
        'Фасоль': 4,
        'Кукуруза': 4,
        'Помидор': 8,
        'Перец': 1,
        'Сыр': 200,
        'Лук': 4,
        'Чеснок': 3,
        'Соль': 2
    },
    
    'Овощи по деревенски': {
        'Картофель': 16,
        'Кабачок': 4,
        'Баклажан': 4,
        'Сельдерей': 1,
        'Помидор': 4,
        'Перец': 8,
        'Петрушка': 2
    }
}

In [ ]:
# Cell #26
def getInsertId(cursor, table, name):
    cursor.execute("SELECT id FROM '%s' WHERE name=?" % table, (name, ))
    results = cursor.fetchall()
    if(len(results) > 0):
        return results[0][0]
    else:
        cursor.execute("INSERT INTO '%s' (name) VALUES (?)" % table, (name, ))
        return getInsertId(cursor, table, name)

In [ ]:
# Cell #27
for dish_name, ingredients in dishes.items():
    for ingredient_name, count in ingredients.items():
        print(ingredient_name, ': ', count, sep='')

In [ ]:
# Cell #28
for dish_name, ingredients in dishes.items():
    for ingredient_name, count in ingredients.items():
        ingredient_id = getInsertId(c, 'ingredients', ingredient_name)
        dish_id = getInsertId(c, 'dishes', dish_name)
        
        c.execute("INSERT INTO dishes_ingredients (dish_id, ingredient_id, ingredient_count) VALUES (?, ?, ?)", 
                       (dish_id, ingredient_id, count))


In [ ]:
# Cell #29
c.execute("""SELECT dishes.name, ingredients.name, ingredient_count FROM dishes_ingredients 
          JOIN dishes ON dish_id = dishes.id
          JOIN ingredients ON ingredient_id = ingredients.id""")
results = c.fetchall()
results

In [ ]:
# Cell #30
# Создаем таблицу для плана блюд
c.execute('''CREATE TABLE dish_plan
             (dish_id INTEGER, count INTEGER)''')

c.execute("INSERT INTO dish_plan (dish_id, count) VALUES (1, 30)")
c.execute("INSERT INTO dish_plan (dish_id, count) VALUES (2, 50)")
c.execute("INSERT INTO dish_plan (dish_id, count) VALUES (3, 70)")

conn.commit()

In [ ]:
# Cell #31
c.execute("""SELECT dishes.name, ingredients.name, ingredient_count, dish_plan.count,
            ingredient_count*dish_plan.count FROM dishes_ingredients 
          JOIN dishes ON dishes_ingredients.dish_id = dishes.id
          JOIN ingredients ON ingredient_id = ingredients.id
          JOIN dish_plan ON dish_plan.dish_id = dishes.id""")
results = c.fetchall()
results

In [ ]:
# Cell #32
c.execute("""SELECT ingredients.name, SUM(ingredient_count*dish_plan.count) FROM dishes_ingredients 
          JOIN dishes ON dishes_ingredients.dish_id = dishes.id
          JOIN ingredients ON ingredient_id = ingredients.id
          JOIN dish_plan ON dish_plan.dish_id = dishes.id
          GROUP BY ingredients.name""")
results = c.fetchall()
results

In [ ]:
# Cell #33
c.execute("SELECT id FROM ingredients")
results = c.fetchall()
ids = list(map(lambda it: it[0], results))
for i in range(1, 28):
    count = random.randint(10, 100)
    date = datetime.datetime(2019, 11, i, hour=random.randint(12, 18), minute=random.randint(0, 59), second=0, microsecond=0, tzinfo=None)
    ingredient = random.choice(ids)
    c.execute("INSERT INTO warehouse (ingredient_id, count, date) VALUES (?, ?, ?)", (ingredient, count, date.timestamp()))

In [ ]:
# Cell #34
def inWarehouse(cursor):
    cursor.execute("""SELECT name, SUM(count) FROM warehouse 
          JOIN ingredients WHERE warehouse.ingredient_id = ingredients.id
          GROUP BY name""")
    return cursor.fetchall()
    
inWarehouse(c)

In [ ]:
# Cell #35
c.execute(""" 
        SELECT name_plan, count_warehouse, count_plan, count_warehouse-count_plan FROM
        (SELECT name as name_warehouse, SUM(count) as count_warehouse FROM warehouse 
          JOIN ingredients WHERE warehouse.ingredient_id = ingredients.id
          GROUP BY name)

        JOIN (
            SELECT ingredients.name as name_plan, 
            SUM(ingredient_count*dish_plan.count) as count_plan
            FROM dishes_ingredients 
            JOIN dishes ON dishes_ingredients.dish_id = dishes.id
            JOIN ingredients ON ingredient_id = ingredients.id
            JOIN dish_plan ON dish_plan.dish_id = dishes.id
            GROUP BY ingredients.name
        ) ON name_warehouse = name_plan """)
results = c.fetchall()
results

In [ ]:
# Cell #36
supply(c, 'Морепродукты', 10, datetime.datetime.now())
prettyWarehouse(c)

In [ ]:
# Cell #37
c.execute("""SELECT name FROM warehouse JOIN ingredients ON warehouse.ingredient_id = ingredients.id
            
            EXCEPT
            
            SELECT ingredients.name FROM dishes_ingredients 
            JOIN dishes ON dish_id = dishes.id
            JOIN ingredients ON ingredient_id = ingredients.id
            
        """)
results = c.fetchall()
results

## Подсказки к домашнему заданию

In [ ]:
# Cell #38
c.execute("SELECT * FROM warehouse ORDER BY date")
results = c.fetchall()
results

In [ ]:
# Cell #39
c.execute("""SELECT name, count, date FROM warehouse 
          JOIN ingredients ON warehouse.ingredient_id = ingredients.id
          ORDER BY date""")
results = c.fetchall()
list(map(lambda it: (it[0], it[1], datetime.datetime.fromtimestamp(it[2]).strftime("%Y-%m-%d %H:%M:%S"))
         , results))

In [ ]:
# Cell #40 
inWarehouse(c)

In [ ]:
# Cell #41
c.execute("""SELECT warehouse.id, count, name FROM warehouse 
          JOIN ingredients ON ingredients.id = warehouse.ingredient_id""")
results = c.fetchall()
results

In [ ]:
# Cell #42
c.execute("UPDATE warehouse SET count = 5 WHERE id=31")
prettyWarehouse(c)

In [ ]:
# Cell #43
inWarehouse(c)

In [ ]:
# Cell #44

# We can also close the connection if we are done with it.
# Just be sure any changes have been committed or they will be lost.
conn.close()

1. Создать таблицу меню, с указанием цен для каждого блюда
2. Создать таблицу чеков, с указанием какие блюда были проданы
3. Создать функцию, которая бы принимала на вход заказ блюд и создавала в бд соответствующий чек и убирала ингредиенты на складе (которые использовались при приготовлении этих блюд)
4. Создать функцию для подсчета выручки (за день/месяц/за всю жизнь)
5. По желанию - продумать архитектуру, чтобы можно было считать прибыль